In [ ]:
# Import local bentoml repository
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), "..", ".."))
import bentoml

print(bentoml.__file__)

# H2O Classification

In [ ]:
# !curl -i https://raw.githubusercontent.com/multicode/h2o-notebook/master/prostate.csv -o prostate.csv

In [ ]:
import h2o

In [ ]:
h2o.init()

In [ ]:
prostate = h2o.import_file(path="https://raw.githubusercontent.com/multicode/h2o-notebook/master/prostate.csv")
prostate.describe()
type(prostate)

In [ ]:
# import the deep learning estimator module
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
# transform the target variable into a factor
prostate["CAPSULE"] = prostate["CAPSULE"].asfactor()
# construct and define the estimator object 
model = H2ODeepLearningEstimator(activation = "Tanh", hidden = [10, 10, 10], epochs = 100)
# train the model on the whole prostate dataset
model.train(x = list(set(prostate.columns) - set(["ID","CAPSULE"])), y ="CAPSULE", training_frame = prostate)
model.show()

In [ ]:
predictions=model.predict(prostate)
predictions.show()
print(type(model))

In [ ]:
path = h2o.save_model(model=model, path="./model")

In [ ]:
print(path)

# Export with BentoML

In [ ]:
%%writefile h2o_model_service.py
import pandas as pd
import h2o
import bentoml
from bentoml.artifact import H2oModelArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([H2oModelArtifact('model')])
@bentoml.env(conda_dependencies=["h2o", "pandas"])
class H2oModelService(bentoml.BentoService):

    @bentoml.api(DataframeHandler)
    def predict(self, df):     
        hf = h2o.H2OFrame(df)
        return self.artifacts.model.predict(hf)

In [ ]:
from h2o_model_service import H2oModelService

# Initialize bentoML model with artifacts

bento_model = H2oModelService.pack(
    model=model
)

# Save bentoML model to directory
saved_path = bento_model.save("/tmp/bento")

# print the directory containing exported model archive (prefixed with model name and version)
print(saved_path)

## Load model

In [ ]:
import bentoml
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/multicode/h2o-notebook/master/prostate.csv")
# Load exported bentoML model archive from path
bento_model = bentoml.load(saved_path)
bento_model.predict(df)